In [ ]:
%matplotlib inline
import ipywidgets
import numpy as np
import matplotlib.pyplot as plt

from os.path import join
from obspy.taup import TauPyModel
import obspy
# %config InlineBackend.figure_format = 'svg'

In [ ]:
model = TauPyModel(model="ak135")

In [ ]:
def init_multiple(network="AH",event="201101060024011",scale=0.5,dir_url="/mnt/research/seismolab2/japan_slab/data/upload_temp_ziyi/extract_cea/extract/processed",minfreq=1/50,maxfreq=1/6,t1=-20,t2=600):
    # urls
    z_url=join(dir_url,event,network+"*Z")
    r_url=join(dir_url,event,network+"*R")
    t_url=join(dir_url,event,network+"*T")
    # read 
    try:    
        z=obspy.read(z_url)
        r=obspy.read(r_url)
        t=obspy.read(t_url)
    except:
        return False,1
    # filter
    z.filter("bandpass",freqmin=minfreq,freqmax=maxfreq)
    r.filter("bandpass",freqmin=minfreq,freqmax=maxfreq)
    t.filter("bandpass",freqmin=minfreq,freqmax=maxfreq)
    
    # fig
    fig = plt.figure(figsize=(18,32))
    axz = fig.add_subplot(3, 1, 1)
    axr = fig.add_subplot(3, 1, 2)
    axt = fig.add_subplot(3, 1, 3)
    
    for index,(wavez,waver,wavet) in enumerate(zip(z,r,t)):
        try:
            assert ((wavez.stats.station==waver.stats.station) and (wavez.stats.station==wavet.stats.station))
        except:
            return False,2
            
        wavez.trim(wavez.stats.starttime+wavez.stats.sac.o,wavez.stats.endtime)
        waver.trim(waver.stats.starttime+waver.stats.sac.o,waver.stats.endtime)
        wavet.trim(wavet.stats.starttime+wavet.stats.sac.o,wavet.stats.endtime)
        
        plot_arrivals={
            "first_p":None,
            "first_s":None,
            "pP":None,
            "PcP":None
        }

        arrivals = model.get_travel_times(source_depth_in_km=wavez.stats.sac.evdp,distance_in_degree=wavez.stats.sac.gcarc)
        for arrival in arrivals:
            if((arrival.name=='p') or (arrival.name=="P")):
                plot_arrivals["first_p"]=arrival.time
                break

        for arrival in arrivals:
            if((arrival.name=='s') or (arrival.name=="S")):
                plot_arrivals["first_s"]=arrival.time
                break

        for arrival in arrivals:
            if((arrival.name=='pP')):
                plot_arrivals["pP"]=arrival.time
                break

        for arrival in arrivals:
            if((arrival.name=='PcP')):
                plot_arrivals["PcP"]=arrival.time
                break
        
        t1_utc=wavez.stats.starttime+plot_arrivals["first_p"]+t1
        end_utc=wavez.stats.starttime+plot_arrivals["first_p"]+t2
        t0_utc=wavez.stats.starttime
        if(t1_utc<t0_utc):
            start_utc=t0_utc
        else:
            start_utc=t1_utc
            
        wavez.trim(start_utc,end_utc)
        waver.trim(start_utc,end_utc)
        wavet.trim(start_utc,end_utc)

        wave_all=obspy.Stream()
        wave_all=wave_all+wavez
        wave_all=wave_all+waver
        wave_all=wave_all+wavet
        wave_all.normalize(global_max=True)
        
        if(t1_utc<t0_utc):
            plotx=np.linspace(-plot_arrivals["first_p"],t2,wavez.stats.npts)
        else:
            plotx=np.linspace(t1,t2,wavez.stats.npts)            
        
        ploty_z=wavez.data*scale+wavez.stats.sac.gcarc
        ploty_r=waver.data*scale+wavez.stats.sac.gcarc
        ploty_t=wavet.data*scale+wavez.stats.sac.gcarc
        
        axz.plot(plotx,ploty_z,"k")
        axr.plot(plotx,ploty_r,"k")
        axt.plot(plotx,ploty_t,"k")
        
        axz.scatter(0,wavez.stats.sac.gcarc,color="r")
        axr.scatter(0,waver.stats.sac.gcarc,color="r")
        axt.scatter(0,wavet.stats.sac.gcarc,color="r")
        
        if(plot_arrivals["first_s"]-plot_arrivals["first_p"]<t2):
            axz.scatter(plot_arrivals["first_s"]-plot_arrivals["first_p"],wavez.stats.sac.gcarc,color="b")
            axr.scatter(plot_arrivals["first_s"]-plot_arrivals["first_p"],waver.stats.sac.gcarc,color="b")
            axt.scatter(plot_arrivals["first_s"]-plot_arrivals["first_p"],wavet.stats.sac.gcarc,color="b")

        try:
            if(plot_arrivals["pP"]-plot_arrivals["first_p"]<t2):
                axz.scatter(plot_arrivals["pP"]-plot_arrivals["first_p"],wavez.stats.sac.gcarc,color="g")
                axr.scatter(plot_arrivals["pP"]-plot_arrivals["first_p"],waver.stats.sac.gcarc,color="g")
                axt.scatter(plot_arrivals["pP"]-plot_arrivals["first_p"],wavet.stats.sac.gcarc,color="g")
            if(plot_arrivals["PcP"]-plot_arrivals["first_p"]<t2):
                axz.scatter(plot_arrivals["PcP"]-plot_arrivals["first_p"],wavez.stats.sac.gcarc,color="purple")
                axr.scatter(plot_arrivals["PcP"]-plot_arrivals["first_p"],waver.stats.sac.gcarc,color="purple")
                axt.scatter(plot_arrivals["PcP"]-plot_arrivals["first_p"],wavet.stats.sac.gcarc,color="purple")
        except:
            pass
    
    axz.set_xlabel("t(s)")
    axz.set_ylabel("distance(degree)")
    axz.set_title(f"{event} {network} z")
    axr.set_xlabel("t(s)")
    axr.set_ylabel("distance(degree)")
    axr.set_title(f"{event} {network} r")
    axt.set_xlabel("t(s)")
    axt.set_ylabel("distance(degree)")
    axt.set_title(f"{event} {network} t")
    plt.show()

In [ ]:
# init_multiple(scale=0.6,network="YN",event="201802061915323",t2=200,maxfreq=1/3)
init_multiple(network="SD",event="201612211644016",scale=0.2)

In [ ]:
def init_single(network="AH",event="201101060024011",station="ANQ",dir_url="/mnt/research/seismolab2/japan_slab/data/upload_temp_ziyi/extract_cea/extract/processed",minfreq=1/50,maxfreq=1/6,t1=-20,t2=600):
    # urls
    z_url=join(dir_url,event,network+"."+station+"*Z")
    r_url=join(dir_url,event,network+"."+station+"*R")
    t_url=join(dir_url,event,network+"."+station+"*T")
    # read 
    try:    
        z=obspy.read(z_url)
        r=obspy.read(r_url)
        t=obspy.read(t_url)
    except:
        return False,1
    # filter
    z.filter("bandpass",freqmin=minfreq,freqmax=maxfreq)
    r.filter("bandpass",freqmin=minfreq,freqmax=maxfreq)
    t.filter("bandpass",freqmin=minfreq,freqmax=maxfreq)
    
    # fig
    fig = plt.figure(figsize=(18,32))
    axz = fig.add_subplot(3, 1, 1)
    axr = fig.add_subplot(3, 1, 2)
    axt = fig.add_subplot(3, 1, 3)
    
    for index,(wavez,waver,wavet) in enumerate(zip(z,r,t)):
        try:
            assert ((wavez.stats.station==waver.stats.station) and (wavez.stats.station==wavet.stats.station))
        except:
            return False,2
            
        wavez.trim(wavez.stats.starttime+wavez.stats.sac.o,wavez.stats.endtime)
        waver.trim(waver.stats.starttime+waver.stats.sac.o,waver.stats.endtime)
        wavet.trim(wavet.stats.starttime+wavet.stats.sac.o,wavet.stats.endtime)
        
        plot_arrivals={
            "first_p":None,
            "first_s":None,
            "pP":None,
            "PcP":None
        }

        arrivals = model.get_travel_times(source_depth_in_km=wavez.stats.sac.evdp,distance_in_degree=wavez.stats.sac.gcarc)
        for arrival in arrivals:
            if((arrival.name=='p') or (arrival.name=="P")):
                plot_arrivals["first_p"]=arrival.time
                break

        for arrival in arrivals:
            if((arrival.name=='s') or (arrival.name=="S")):
                plot_arrivals["first_s"]=arrival.time
                break

        for arrival in arrivals:
            if((arrival.name=='pP')):
                plot_arrivals["pP"]=arrival.time
                break

        for arrival in arrivals:
            if((arrival.name=='PcP')):
                plot_arrivals["PcP"]=arrival.time
                break
        
        t1_utc=wavez.stats.starttime+plot_arrivals["first_p"]+t1
        end_utc=wavez.stats.starttime+plot_arrivals["first_p"]+t2
        t0_utc=wavez.stats.starttime
        print("starttime",wavez.stats.starttime)
        print("p arrival",wavez.stats.starttime+plot_arrivals["first_p"])
        print("s arrival",wavez.stats.starttime+plot_arrivals["first_s"])
        if(t1_utc<t0_utc):
            start_utc=t0_utc
        else:
            start_utc=t1_utc
            
        wavez.trim(start_utc,end_utc)
        waver.trim(start_utc,end_utc)
        wavet.trim(start_utc,end_utc)

#         wave_all=obspy.Stream()
#         wave_all=wave_all+wavez
#         wave_all=wave_all+waver
#         wave_all=wave_all+wavet
#         wave_all.normalize(global_max=True)
#         wavez.normalize()
#         waver.normalize()
#         wavet.normalize()
        
        if(t1_utc<t0_utc):
            plotx=np.linspace(-plot_arrivals["first_p"],t2,wavez.stats.npts)
        else:
            plotx=np.linspace(t1,t2,wavez.stats.npts)            
        
        ploty_z=wavez.data
        ploty_r=waver.data
        ploty_t=wavet.data
        
        axz.plot(plotx,ploty_z,"k")
        axr.plot(plotx,ploty_r,"k")
        axt.plot(plotx,ploty_t,"k")
        
        axz.scatter(0,wavez.stats.sac.gcarc,color="r",label="first p")
        axr.scatter(0,waver.stats.sac.gcarc,color="r",label="first p")
        axt.scatter(0,wavet.stats.sac.gcarc,color="r",label="first p")
        
        if(plot_arrivals["first_s"]-plot_arrivals["first_p"]<t2):
            axz.scatter(plot_arrivals["first_s"]-plot_arrivals["first_p"],wavez.stats.sac.gcarc,color="b",label="first s")
            axr.scatter(plot_arrivals["first_s"]-plot_arrivals["first_p"],waver.stats.sac.gcarc,color="b",label="first s")
            axt.scatter(plot_arrivals["first_s"]-plot_arrivals["first_p"],wavet.stats.sac.gcarc,color="b",label="first s")

        try:
            if(plot_arrivals["pP"]-plot_arrivals["first_p"]<t2):
                axz.scatter(plot_arrivals["pP"]-plot_arrivals["first_p"],wavez.stats.sac.gcarc,color="g",label="pP")
                axr.scatter(plot_arrivals["pP"]-plot_arrivals["first_p"],waver.stats.sac.gcarc,color="g",label="pP")
                axt.scatter(plot_arrivals["pP"]-plot_arrivals["first_p"],wavet.stats.sac.gcarc,color="g",label="pP")
            if(plot_arrivals["PcP"]-plot_arrivals["first_p"]<t2):
                axz.scatter(plot_arrivals["PcP"]-plot_arrivals["first_p"],wavez.stats.sac.gcarc,color="purple",label="PnP")
                axr.scatter(plot_arrivals["PcP"]-plot_arrivals["first_p"],waver.stats.sac.gcarc,color="purple",label="PnP")
                axt.scatter(plot_arrivals["PcP"]-plot_arrivals["first_p"],wavet.stats.sac.gcarc,color="purple",label="PnP")
        except:
            pass
    
    axz.set_xlabel("t(s)")
    axz.set_ylabel("amplitude")
    axz.set_title(f"{event} {network} {station} z")
    axz.legend()
    axr.set_xlabel("t(s)")
    axr.set_ylabel("distance(degree)")
    axr.set_title(f"{event} {network} {station} r")
    axr.legend()
    axt.set_xlabel("t(s)")
    axt.set_ylabel("distance(degree)")
    axt.set_title(f"{event} {network} {station} t")
    axt.legend()
    plt.show()

In [ ]:
# /mnt/research/seismolab2/japan_slab/data/upload_temp_ziyi/extract_cea/extract/processed/201506042315466/NM.TIS.00
ploturl="/mnt/research/seismolab2/japan_slab/data/upload_temp_ziyi/extract_cea/extract/processed/201208300656433/JL.SYZT.00..Z"
init_single(event=ploturl.split("/")[-2],station=ploturl.split("/")[-1].split(".")[1],network=ploturl.split("/")[-1].split(".")[0],t2=3*60,t1=-3*60)
init_single(event=ploturl.split("/")[-2],station=ploturl.split("/")[-1].split(".")[1],network=ploturl.split("/")[-1].split(".")[0],t2=45*60,t1=-3*60)
obspy.read(ploturl).filter("bandpass",freqmin=1/50.,freqmax=1/6.).plot()
obspy.read(ploturl)[0].stats